# Result Extraction - Code Laboratory

**Section 7: Result Extraction (10% of Exam)** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

### Sampler Result Access (RIDMG Pattern)
| Method | Returns | Use Case |
|--------|---------|----------|
| `result[0].data.meas.get_counts()` | `dict[str, int]` | Primary - counts by bitstring |
| `result[0].data.meas.get_bitstrings()` | `list[str]` | Individual shot outcomes |
| `result[0].data.meas.get_int_counts()` | `dict[int, int]` | Counts with integer keys |

### Estimator Result Access (EVS = plural, property)
| Property | Returns | Use Case |
|----------|---------|----------|
| `result[0].data.evs` | `float` | Expectation value (no parentheses!) |
| `result[0].data.stds` | `float` | Standard deviation |

### PUB Formats (Input Structure)
| Primitive | PUB Format | Mnemonic |
|-----------|------------|----------|
| Sampler | `(circuit, params, shots)` | **CPS** |
| Estimator | `(circuit, observable, params, precision)` | **COPP** |

### Job & Metadata
| Access | Returns | Use Case |
|--------|---------|----------|
| `result[0].metadata` | `dict` | Execution information |
| `job.status()` | `JobStatus` | Job lifecycle state |
| `service.jobs(...)` | `list[Job]` | Job history filtering |

## 🧠 Mnemonics
- **RIDMG**: Result → Index → Data → Meas → Get_counts()
- **EVS**: Expectation ValueS (plural, property not method!)
- **IQVRCDE**: Initializing, Queued, Validating, Running, Cancelled, Done, Error

---

In [ ]:
# ============================================================
# ENVIRONMENT SETUP - Run this first!
# ============================================================

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.primitives import StatevectorSampler, StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Aliases for cleaner code
Sampler = StatevectorSampler
Estimator = StatevectorEstimator

print("✅ Environment ready")
print("   Sampler: result[0].data.meas.get_counts()")
print("   Estimator: result[0].data.evs")
print("   Mnemonic: RIDMG - Result Index Data Meas Get")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 1: SAMPLER RESULT STRUCTURE (RIDMG Mnemonic)
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 RIDMG = Result Index Data Meas Get
# result[0].data.meas.get_counts()

# Create Bell state circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

print("Bell State Circuit:")
print(qc.draw())

# Run with Sampler
job = sampler.run([qc], shots=1024)
result = job.result()

# ══════════════════════════════════════════════════════════════════════════════
# RESULT STRUCTURE HIERARCHY (follow RIDMG mnemonic)
# ══════════════════════════════════════════════════════════════════════════════
print("\n📊 RESULT STRUCTURE:")
print(f"  result type:           {type(result).__name__}")
print(f"  result[0] type:        {type(result[0]).__name__}")
print(f"  result[0].data type:   {type(result[0].data).__name__}")
print(f"  result[0].data.meas:   {type(result[0].data.meas).__name__}")

# ══════════════════════════════════════════════════════════════════════════════
# THE EXAM LINE - MEMORIZE THIS!
# ══════════════════════════════════════════════════════════════════════════════
counts = result[0].data.meas.get_counts()  # ← THE PATTERN!

print(f"\n🎯 EXAM LINE: counts = result[0].data.meas.get_counts()")
print(f"   Result: {counts}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION - Bell state should have ~50% |00⟩, ~50% |11⟩
# ══════════════════════════════════════════════════════════════════════════════
assert isinstance(counts, dict), "❌ get_counts() should return dict!"
assert sum(counts.values()) == 1024, "❌ Total should equal shots!"
assert '00' in counts or '11' in counts, "❌ Bell state missing expected outcomes!"

print("\n✅ All assertions passed! RIDMG pattern works.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 2: GET_COUNTS() - DICTIONARY OPERATIONS
# ═══════════════════════════════════════════════════════════════════════════════
# Returns: dict[str, int] - {bitstring: count}

# Create 2-qubit superposition (all 4 states equally likely)
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.h(1)
qc.measure([0, 1], [0, 1])

job = sampler.run([qc], shots=1000)
result = job.result()

# ══════════════════════════════════════════════════════════════════════════════
# EXTRACT COUNTS - THE EXAM LINE
# ══════════════════════════════════════════════════════════════════════════════
counts = result[0].data.meas.get_counts()

print("📊 GET_COUNTS() RESULT:")
print(f"   Type: {type(counts).__name__}")
print(f"   Value: {counts}")

# ══════════════════════════════════════════════════════════════════════════════
# COMMON DICT OPERATIONS (frequently tested)
# ══════════════════════════════════════════════════════════════════════════════
# 1. Find most frequent outcome
max_state = max(counts, key=counts.get)
print(f"\n🔹 Most frequent: '{max_state}' → {counts[max_state]} counts")

# 2. Convert to probabilities
total = sum(counts.values())
probs = {k: v/total for k, v in counts.items()}
print(f"🔹 Probabilities: {probs}")

# 3. Check if specific state exists
has_00 = '00' in counts
print(f"🔹 Has '00' state: {has_00}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
assert isinstance(counts, dict), "❌ counts should be dict!"
assert all(isinstance(k, str) for k in counts.keys()), "❌ keys should be strings!"
assert all(isinstance(v, int) for v in counts.values()), "❌ values should be ints!"
assert abs(sum(probs.values()) - 1.0) < 0.001, "❌ probabilities should sum to 1!"

print("\n✅ All assertions passed!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 3: GET_BITSTRINGS() - INDIVIDUAL SHOT RESULTS
# ═══════════════════════════════════════════════════════════════════════════════
# Returns: list[str] - one bitstring per shot

# Bell state - entangled qubits
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

job = sampler.run([qc], shots=100)  # Small for visibility
result = job.result()

# ══════════════════════════════════════════════════════════════════════════════
# GET_BITSTRINGS() - LIST OF INDIVIDUAL OUTCOMES
# ══════════════════════════════════════════════════════════════════════════════
bitstrings = result[0].data.meas.get_bitstrings()

print("📊 GET_BITSTRINGS() RESULT:")
print(f"   Type: {type(bitstrings).__name__}")
print(f"   Length: {len(bitstrings)} (equals shots)")
print(f"   First 10: {bitstrings[:10]}")

# ══════════════════════════════════════════════════════════════════════════════
# COMPARE: get_counts() vs get_bitstrings()
# ══════════════════════════════════════════════════════════════════════════════
counts = result[0].data.meas.get_counts()

print("\n📌 COMPARISON:")
print(f"   get_counts():     {counts}  ← dict")
print(f"   get_bitstrings(): {bitstrings[:5]}...  ← list")

# ══════════════════════════════════════════════════════════════════════════════
# CONVERT BITSTRINGS TO COUNTS
# ══════════════════════════════════════════════════════════════════════════════
from collections import Counter
manual_counts = dict(Counter(bitstrings))
print(f"\n🔹 Counter(bitstrings) = {manual_counts}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
assert isinstance(bitstrings, list), "❌ bitstrings should be list!"
assert len(bitstrings) == 100, "❌ length should equal shots!"
assert all(b in ['00', '11'] for b in bitstrings), "❌ Bell state should only have 00/11!"

print("\n✅ All assertions passed!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 3B: GET_INT_COUNTS() - INTEGER KEY FORMAT
# ═══════════════════════════════════════════════════════════════════════════════
# Returns: dict[int, int] - {integer_value: count}
# Converts bitstrings to their integer values

# Bell state for demonstration
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

job = sampler.run([qc], shots=1000)
result = job.result()

# ══════════════════════════════════════════════════════════════════════════════
# COMPARE ALL THREE METHODS
# ══════════════════════════════════════════════════════════════════════════════
counts = result[0].data.meas.get_counts()
int_counts = result[0].data.meas.get_int_counts()
bitstrings = result[0].data.meas.get_bitstrings()

print("📊 THREE RESULT EXTRACTION METHODS:")
print("="*60)
print(f"\n🔹 get_counts():")
print(f"   Returns: dict[str, int]")
print(f"   Value: {counts}")

print(f"\n🔹 get_int_counts():")
print(f"   Returns: dict[int, int]")
print(f"   Value: {int_counts}")
print(f"   Mapping: '00' → 0, '01' → 1, '10' → 2, '11' → 3")

print(f"\n🔹 get_bitstrings():")
print(f"   Returns: list[str]")
print(f"   Length: {len(bitstrings)} (equals shots)")
print(f"   First 5: {bitstrings[:5]}")

# ══════════════════════════════════════════════════════════════════════════════
# BITSTRING TO INTEGER CONVERSION
# ══════════════════════════════════════════════════════════════════════════════
print("\n📌 BITSTRING → INTEGER CONVERSION:")
print("   '00' → int('00', 2) = 0  (0×2¹ + 0×2⁰)")
print("   '01' → int('01', 2) = 1  (0×2¹ + 1×2⁰)")
print("   '10' → int('10', 2) = 2  (1×2¹ + 0×2⁰)")
print("   '11' → int('11', 2) = 3  (1×2¹ + 1×2⁰)")

# Demonstrate conversion
for bs, count in counts.items():
    int_val = int(bs, 2)
    print(f"   '{bs}' → {int_val}: {count} counts")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
# Bell state should have 0 and 3 (i.e., '00' and '11')
assert 0 in int_counts or 3 in int_counts, "❌ Bell state should have 0 or 3!"
assert isinstance(list(int_counts.keys())[0], int), "❌ Keys should be integers!"

print("\n✅ All assertions passed! get_int_counts() works correctly.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 4: ESTIMATOR RESULT STRUCTURE (EVS Mnemonic)
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 EVS = Expectation ValueS (PLURAL - no parentheses!)
# result[0].data.evs  ← property, NOT method!

# Create Bell circuit (NO measurements for Estimator!)
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
# NOTE: NO .measure_all() - Estimator handles measurement internally!

print("Bell State Circuit (NO measurements):")
print(qc.draw())

# Define observable - ZZ measures correlation
observable = SparsePauliOp(['ZZ'])

# ══════════════════════════════════════════════════════════════════════════════
# RUN ESTIMATOR
# ══════════════════════════════════════════════════════════════════════════════
job = estimator.run([(qc, observable)])  # Note: tuple in list
result = job.result()

print("\n📊 ESTIMATOR RESULT STRUCTURE:")
print(f"   result type:           {type(result).__name__}")
print(f"   result[0] type:        {type(result[0]).__name__}")
print(f"   result[0].data type:   {type(result[0].data).__name__}")

# ══════════════════════════════════════════════════════════════════════════════
# THE EXAM LINE - MEMORIZE THIS!
# ══════════════════════════════════════════════════════════════════════════════
expectation = result[0].data.evs  # ← PROPERTY, no ()!
std_error = result[0].data.stds   # ← Standard deviation

print(f"\n🎯 EXAM LINE: expectation = result[0].data.evs")
print(f"   Expectation Value: {expectation}")
print(f"   Standard Error:    {std_error}")

# ══════════════════════════════════════════════════════════════════════════════
# ⚠️ CRITICAL TRAP: .evs is PROPERTY, not method!
# ══════════════════════════════════════════════════════════════════════════════
print("\n⚠️ TRAP ALERT:")
print("   ✅ result[0].data.evs      ← CORRECT (property)")
print("   ❌ result[0].data.evs()    ← WRONG (not callable!)")
print("   ❌ result[0].data.ev       ← WRONG (it's evs - plural!)")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
# Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2 has ⟨ZZ⟩ = 1
assert expectation > 0.9, f"❌ Bell state ZZ expectation should be ~1, got {expectation}"
assert not callable(result[0].data.evs), "❌ .evs should NOT be callable!"

print("\n✅ All assertions passed! EVS pattern works.")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 5: MULTIPLE OBSERVABLES WITH ESTIMATOR
# ═══════════════════════════════════════════════════════════════════════════════
# Each (circuit, observable) pair → separate PubResult at result[i]

# Bell state circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)

# Multiple observables for Bell state
observables = [
    SparsePauliOp(['ZZ']),  # Correlation in Z basis
    SparsePauliOp(['XX']),  # Correlation in X basis
    SparsePauliOp(['YY'])   # Correlation in Y basis
]

# ══════════════════════════════════════════════════════════════════════════════
# RUN WITH MULTIPLE OBSERVABLES
# ══════════════════════════════════════════════════════════════════════════════
# Create list of (circuit, observable) tuples
pubs = [(qc, obs) for obs in observables]
job = estimator.run(pubs)
result = job.result()

print("📊 MULTIPLE OBSERVABLE RESULTS:")
print(f"   Number of PubResults: {len(result)}")

# ══════════════════════════════════════════════════════════════════════════════
# ACCESS EACH RESULT BY INDEX
# ══════════════════════════════════════════════════════════════════════════════
obs_names = ['ZZ', 'XX', 'YY']
results_dict = {}

print("\n🔹 Bell State Expectation Values:")
for i, obs_name in enumerate(obs_names):
    ev = result[i].data.evs  # Index into result!
    results_dict[obs_name] = ev
    print(f"   ⟨{obs_name}⟩ = {float(ev):.4f}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION - Bell state |Φ+⟩ signature
# ══════════════════════════════════════════════════════════════════════════════
# Bell state |Φ+⟩ = (|00⟩ + |11⟩)/√2 has:
#   ⟨ZZ⟩ = +1, ⟨XX⟩ = +1, ⟨YY⟩ = -1
assert len(result) == 3, "❌ Should have 3 PubResults!"
assert float(results_dict['ZZ']) > 0.9, "❌ ZZ should be ~+1"
assert float(results_dict['XX']) > 0.9, "❌ XX should be ~+1"
assert float(results_dict['YY']) < -0.9, "❌ YY should be ~-1"

print("\n✅ All assertions passed! Bell state signature verified.")

---

## PUB Format (Primitive Unified Bloc)

### Sampler PUB: `(circuit, params, shots)`
| Element | Type | Required | Description |
|---------|------|----------|-------------|
| `circuit` | `QuantumCircuit` | Yes | **MUST have measurements** |
| `params` | `list` | No | Parameter values for parameterized circuits |
| `shots` | `int` | No | Number of shots (default varies) |

### Estimator PUB: `(circuit, observable, params, precision)`
| Element | Type | Required | Description |
|---------|------|----------|-------------|
| `circuit` | `QuantumCircuit` | Yes | **MUST NOT have measurements** |
| `observable` | `SparsePauliOp` | Yes | Observable to measure (NOT a string!) |
| `params` | `list` | No | Parameter values |
| `precision` | `float` | No | Target precision |

### ⚠️ Critical Traps
- Trailing comma for single tuple: `[(circuit,)]` not `[(circuit)]`
- Observable must be `SparsePauliOp`, not string `'ZZ'`

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 5B: PUB FORMAT DEMONSTRATIONS
# ═══════════════════════════════════════════════════════════════════════════════
# PUB = Primitive Unified Bloc - the input structure for primitives

print("📊 PUB FORMAT DEMONSTRATIONS")
print("="*60)

# ══════════════════════════════════════════════════════════════════════════════
# SAMPLER PUB: (circuit, params, shots) - "CPS"
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 SAMPLER PUB FORMAT: (circuit, params, shots)")
print("-"*50)

# Basic PUB - just circuit (needs trailing comma!)
qc_basic = QuantumCircuit(1)
qc_basic.h(0)
qc_basic.measure_all()

print("1. Basic: [(circuit,)] - trailing comma required!")
job1 = sampler.run([(qc_basic,)], shots=100)
print(f"   Result: {job1.result()[0].data.meas.get_counts()}")

# With parameters
theta = Parameter('θ')
qc_param = QuantumCircuit(1)
qc_param.ry(theta, 0)
qc_param.measure_all()

print("\n2. With params: [(circuit, [param_values])]")
job2 = sampler.run([(qc_param, [np.pi/2])], shots=100)  # θ = π/2
print(f"   Result: {job2.result()[0].data.meas.get_counts()}")

# ══════════════════════════════════════════════════════════════════════════════
# ESTIMATOR PUB: (circuit, observable, params, precision) - "COPP"
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 ESTIMATOR PUB FORMAT: (circuit, observable, params, precision)")
print("-"*50)

# Basic PUB - circuit and observable (both required!)
qc_est = QuantumCircuit(1)
qc_est.h(0)  # NO measurements!

print("1. Basic: [(circuit, SparsePauliOp('Z'))]")
job3 = estimator.run([(qc_est, SparsePauliOp('Z'))])
print(f"   Result: ⟨Z⟩ = {job3.result()[0].data.evs:.4f}")

# With parameters
qc_param_est = QuantumCircuit(1)
qc_param_est.ry(theta, 0)

print("\n2. With params: [(circuit, observable, [param_values])]")
job4 = estimator.run([(qc_param_est, SparsePauliOp('Z'), [np.pi])])  # θ = π → |1⟩
print(f"   Result: ⟨Z⟩ = {job4.result()[0].data.evs:.4f} (should be -1)")

# ══════════════════════════════════════════════════════════════════════════════
# ⚠️ PUB TRAPS
# ══════════════════════════════════════════════════════════════════════════════
print("\n⚠️ PUB FORMAT TRAPS:")
print("-"*50)

print("❌ WRONG: sampler.run([circuit])        # Not a tuple!")
print("❌ WRONG: sampler.run([(circuit)])      # Missing trailing comma!")
print("✅ CORRECT: sampler.run([(circuit,)])   # Trailing comma!")

print("\n❌ WRONG: estimator.run([(circuit, 'ZZ')])  # String not SparsePauliOp!")
print("✅ CORRECT: estimator.run([(circuit, SparsePauliOp('ZZ'))])")

print("\n✅ PUB format demonstrations complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 6: MULTIPLE CIRCUITS - BATCH PROCESSING
# ═══════════════════════════════════════════════════════════════════════════════
# Each circuit in list → separate PubResult at result[i]

# Create 3 different circuits
qc_h = QuantumCircuit(1, name='H')
qc_h.h(0)
qc_h.measure_all()

qc_x = QuantumCircuit(1, name='X')
qc_x.x(0)
qc_x.measure_all()

qc_i = QuantumCircuit(1, name='I')
qc_i.measure_all()  # Identity → |0⟩

# ══════════════════════════════════════════════════════════════════════════════
# RUN BATCH - Pass list of circuits
# ══════════════════════════════════════════════════════════════════════════════
circuits = [qc_h, qc_x, qc_i]
job = sampler.run(circuits, shots=1000)
result = job.result()

print("📊 BATCH PROCESSING RESULTS:")
print(f"   Number of PubResults: {len(result)}")

# ══════════════════════════════════════════════════════════════════════════════
# METHOD 1: Loop through results
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 METHOD 1 - Loop through enumerate(result):")
for i, pub_result in enumerate(result):
    counts = pub_result.data.meas.get_counts()
    print(f"   Circuit {i}: {counts}")

# ══════════════════════════════════════════════════════════════════════════════
# METHOD 2: Direct indexing
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 METHOD 2 - Direct indexing:")
counts_0 = result[0].data.meas.get_counts()  # H gate
counts_1 = result[1].data.meas.get_counts()  # X gate
counts_2 = result[2].data.meas.get_counts()  # Identity
print(f"   result[0] (H):   {counts_0}")
print(f"   result[1] (X):   {counts_1}")
print(f"   result[2] (I):   {counts_2}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
# H gate: ~50% |0⟩, ~50% |1⟩
assert '0' in counts_0 and '1' in counts_0, "❌ H gate should produce 0 and 1"
# X gate: 100% |1⟩
assert counts_1.get('1', 0) == 1000, "❌ X gate should produce only 1s"
# Identity: 100% |0⟩
assert counts_2.get('0', 0) == 1000, "❌ Identity should produce only 0s"

print("\n✅ All assertions passed!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 6B: METADATA ACCESS
# ═══════════════════════════════════════════════════════════════════════════════
# result[0].metadata contains execution information

print("📊 METADATA ACCESS")
print("="*60)

# Create and run a circuit
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

job = sampler.run([qc], shots=1000)
result = job.result()

# ══════════════════════════════════════════════════════════════════════════════
# ACCESS METADATA
# ══════════════════════════════════════════════════════════════════════════════
metadata = result[0].metadata

print("\n🔹 Metadata Access Pattern:")
print("   metadata = result[0].metadata")
print(f"\n🔹 Metadata Type: {type(metadata).__name__}")
print(f"🔹 Metadata Content: {metadata}")

# ══════════════════════════════════════════════════════════════════════════════
# COMMON METADATA FIELDS
# ══════════════════════════════════════════════════════════════════════════════
print("\n📌 Common Metadata Fields (varies by backend):")
print("-"*50)

# Try to access common fields safely
if 'shots' in metadata:
    print(f"   shots: {metadata['shots']}")
if 'circuit_metadata' in metadata:
    print(f"   circuit_metadata: {metadata['circuit_metadata']}")

# Show all available keys
print(f"\n   Available keys: {list(metadata.keys())}")

# ══════════════════════════════════════════════════════════════════════════════
# VERIFICATION
# ══════════════════════════════════════════════════════════════════════════════
assert isinstance(metadata, dict), "❌ Metadata should be a dict!"
print("\n✅ Metadata access works correctly!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 7: ⚠️ TRAP DEMONSTRATION - COMMON EXAM MISTAKES
# ═══════════════════════════════════════════════════════════════════════════════

print("⚠️ TRAP DEMONSTRATION - AVOID THESE MISTAKES!\n")

# ══════════════════════════════════════════════════════════════════════════════
# TRAP 1: Missing [0] index
# ══════════════════════════════════════════════════════════════════════════════
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.measure_all()

job = sampler.run([qc], shots=100)
result = job.result()

print("TRAP 1: Missing [0] index")
print("   ❌ result.data.meas.get_counts()  → AttributeError!")
print("   ✅ result[0].data.meas.get_counts()")
try:
    wrong = result.data  # Will fail
except AttributeError as e:
    print(f"   Error: {type(e).__name__}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAP 2: .evs() with parentheses (it's a PROPERTY!)
# ══════════════════════════════════════════════════════════════════════════════
qc_est = QuantumCircuit(2)
qc_est.h(0)
qc_est.cx(0, 1)

job = estimator.run([(qc_est, SparsePauliOp(['ZZ']))])
result = job.result()

print("\nTRAP 2: .evs() with parentheses")
print("   ❌ result[0].data.evs()  → TypeError: not callable!")
print("   ✅ result[0].data.evs    ← property, no parentheses")

# Correct way:
correct_evs = result[0].data.evs
print(f"   Correct result: {correct_evs}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAP 3: .ev instead of .evs (SINGULAR vs PLURAL)
# ══════════════════════════════════════════════════════════════════════════════
print("\nTRAP 3: .ev vs .evs (singular vs plural)")
print("   ❌ result[0].data.ev   → AttributeError!")
print("   ✅ result[0].data.evs  ← PLURAL!")
print("   ❌ result[0].data.std  → AttributeError!")
print("   ✅ result[0].data.stds ← PLURAL!")

# ══════════════════════════════════════════════════════════════════════════════
# TRAP 4: .counts() instead of .get_counts()
# ══════════════════════════════════════════════════════════════════════════════
print("\nTRAP 4: .counts() vs .get_counts()")
print("   ❌ result[0].data.meas.counts()      → AttributeError!")
print("   ✅ result[0].data.meas.get_counts()  ← include 'get_'")

# ══════════════════════════════════════════════════════════════════════════════
# TRAP 5: Wrong register name
# ══════════════════════════════════════════════════════════════════════════════
print("\nTRAP 5: Wrong classical register name")
# Default measure_all() creates 'meas' register
# But explicit measure can create different names!
qc_custom = QuantumCircuit(2, 2)
qc_custom.h(0)
qc_custom.measure([0, 1], [0, 1])  # Uses default 'c' register!

job = sampler.run([qc_custom], shots=100)
result = job.result()

# Check what registers exist
data_bin = result[0].data
print(f"   Available registers: {dir(data_bin)}")
# 'c' is the default classical register name, 'meas' is from measure_all()

print("\n" + "="*70)
print("🎯 SUMMARY - MEMORIZE THE CORRECT PATTERNS:")
print("="*70)
print("   Sampler:   result[0].data.meas.get_counts()")
print("   Estimator: result[0].data.evs  ← no parentheses!")
print("="*70)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 8: JOBSTATUS ENUMERATION (IQVRCDE Mnemonic)
# ═══════════════════════════════════════════════════════════════════════════════
# 🎯 IQVRCDE = Initializing, Queued, Validating, Running, Cancelled, Done, Error
# from qiskit.providers import JobStatus

print("📊 JOBSTATUS ENUMERATION:")
print("="*60)

# All JobStatus values (memorize order!)
statuses = [
    ("INITIALIZING", "Setting up job"),
    ("QUEUED", "Waiting in queue - can cancel"),
    ("VALIDATING", "Being validated"),
    ("RUNNING", "Currently executing - cannot cancel"),
    ("CANCELLED", "User cancelled"),
    ("DONE", "Successfully completed ✅"),
    ("ERROR", "Failed with error ❌")
]

print("\n🔹 JobStatus Values (IQVRCDE):")
for status, description in statuses:
    print(f"   JobStatus.{status:14} - {description}")

# ══════════════════════════════════════════════════════════════════════════════
# NORMAL JOB LIFECYCLE
# ══════════════════════════════════════════════════════════════════════════════
print("\n📈 Normal Job Lifecycle:")
print("   INITIALIZING → QUEUED → VALIDATING → RUNNING → DONE")

# ══════════════════════════════════════════════════════════════════════════════
# ⚠️ CRITICAL TRAP: Compare with ENUM, not strings!
# ══════════════════════════════════════════════════════════════════════════════
print("\n⚠️ TRAP: Compare with ENUM, not strings!")
print("   ✅ if job.status() == JobStatus.DONE:")
print("   ❌ if job.status() == 'DONE':  # WRONG!")

# ══════════════════════════════════════════════════════════════════════════════
# DEMONSTRATE WITH LOCAL JOB (StatevectorSampler always returns DONE)
# ══════════════════════════════════════════════════════════════════════════════
qc = QuantumCircuit(1)
qc.h(0)
qc.measure_all()

job = sampler.run([qc])
print(f"\n🔹 Local job status: {job.status()}")
print(f"   Type: {type(job.status())}")

# Verify correct comparison
from qiskit.providers import JobStatus
assert job.status() == JobStatus.DONE, "❌ Job should be DONE!"
print("\n✅ JobStatus comparison works correctly!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LAB 8B: SERVICE.JOBS() AND RUNTIMEENCODER (IBM QUANTUM ONLY)
# ═══════════════════════════════════════════════════════════════════════════════
# These features require IBM Quantum Runtime - demonstrated as reference patterns

print("📊 IBM QUANTUM JOB MANAGEMENT (Reference Patterns)")
print("="*60)

# ══════════════════════════════════════════════════════════════════════════════
# SERVICE.JOBS() - Job History Filtering
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 service.jobs() - Job History Retrieval")
print("-"*50)

service_jobs_example = '''
from qiskit_ibm_runtime import QiskitRuntimeService
from datetime import datetime, timedelta

# Connect to IBM Quantum
service = QiskitRuntimeService()

# Get last 10 jobs (default)
jobs = service.jobs()

# Filter by limit
jobs = service.jobs(limit=50)

# Filter by backend
jobs = service.jobs(backend_name="ibm_brisbane")

# Filter by pending status
jobs = service.jobs(pending=True)   # Incomplete jobs
jobs = service.jobs(pending=False)  # Completed jobs

# Filter by time
start = datetime.now() - timedelta(days=7)
jobs = service.jobs(created_after=start)

# Combined filters
jobs = service.jobs(
    backend_name="ibm_brisbane",
    pending=False,
    limit=20,
    created_after=start
)
'''
print(service_jobs_example)

# ══════════════════════════════════════════════════════════════════════════════
# SERVICE.JOBS() PARAMETERS
# ══════════════════════════════════════════════════════════════════════════════
print("📌 service.jobs() Parameters:")
print("-"*50)
params = [
    ("limit", "int", "Max jobs to return (default: 10)"),
    ("skip", "int", "Jobs to skip (pagination)"),
    ("backend_name", "str", "Filter by backend"),
    ("pending", "bool", "True=incomplete, False=completed"),
    ("created_after", "datetime", "Jobs after this time"),
    ("created_before", "datetime", "Jobs before this time"),
    ("program_id", "str", "Filter by program"),
    ("job_tags", "list[str]", "Filter by tags"),
]
for param, typ, desc in params:
    print(f"   {param:15} ({typ:12}): {desc}")

# ══════════════════════════════════════════════════════════════════════════════
# RUNTIMEENCODER/RUNTIMEDECODER
# ══════════════════════════════════════════════════════════════════════════════
print("\n🔹 RuntimeEncoder/RuntimeDecoder - Serialization")
print("-"*50)

encoder_example = '''
import json
from qiskit_ibm_runtime import RuntimeEncoder, RuntimeDecoder
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp

# Create objects to serialize
qc = QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
obs = SparsePauliOp('ZZ')

# Encode to JSON
data = {'circuit': qc, 'observable': obs}
encoded = json.dumps(data, cls=RuntimeEncoder)

# Decode from JSON
recovered = json.loads(encoded, cls=RuntimeDecoder)
'''
print(encoder_example)

print("\n📌 When to use RuntimeEncoder/Decoder:")
print("   ✅ Custom Runtime Programs")
print("   ✅ REST API calls to IBM Quantum")
print("   ✅ Saving/loading Qiskit objects to JSON")
print("   ❌ Standard primitives API (automatic)")

print("\n✅ IBM Quantum job management patterns complete!")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# 🏆 CODE CHALLENGES - VERIFY YOUR KNOWLEDGE
# ═══════════════════════════════════════════════════════════════════════════════
# Fill in the blanks and run to verify!

print("🏆 CODE CHALLENGES\n")

# ══════════════════════════════════════════════════════════════════════════════
# CHALLENGE 1: Extract Sampler counts
# ══════════════════════════════════════════════════════════════════════════════
print("CHALLENGE 1: Extract Sampler counts")
qc1 = QuantumCircuit(2)
qc1.h(0)
qc1.cx(0, 1)
qc1.measure_all()

job1 = sampler.run([qc1], shots=500)
result1 = job1.result()

# TODO: Fill in the correct access pattern (follow RIDMG mnemonic)
counts1 = result1[0].data.meas.get_counts()  # ← Verify this is correct!

assert isinstance(counts1, dict), "❌ Challenge 1 failed: counts should be dict!"
assert sum(counts1.values()) == 500, "❌ Challenge 1 failed: total should be 500!"
print(f"   ✅ Counts: {counts1}")

# ══════════════════════════════════════════════════════════════════════════════
# CHALLENGE 2: Extract Estimator expectation value
# ══════════════════════════════════════════════════════════════════════════════
print("\nCHALLENGE 2: Extract Estimator expectation value")
qc2 = QuantumCircuit(1)
qc2.x(0)  # |1⟩ state

job2 = estimator.run([(qc2, SparsePauliOp(['Z']))])
result2 = job2.result()

# TODO: Fill in the correct access pattern (remember: no parentheses!)
evs2 = result2[0].data.evs  # ← Verify this is correct!

# |1⟩ state has ⟨Z⟩ = -1
assert float(evs2) < -0.9, f"❌ Challenge 2 failed: |1⟩ should have ⟨Z⟩ ≈ -1, got {evs2}"
print(f"   ✅ ⟨Z⟩ = {float(evs2):.4f}")

# ══════════════════════════════════════════════════════════════════════════════
# CHALLENGE 3: Process multiple circuits
# ══════════════════════════════════════════════════════════════════════════════
print("\nCHALLENGE 3: Process multiple circuits")
circuits = []
for i in range(3):
    qc = QuantumCircuit(1)
    if i == 0: qc.h(0)   # Superposition
    if i == 1: qc.x(0)   # |1⟩
    # i == 2: Identity → |0⟩
    qc.measure_all()
    circuits.append(qc)

job3 = sampler.run(circuits, shots=100)
result3 = job3.result()

# TODO: Extract counts for all 3 circuits
all_counts = [result3[i].data.meas.get_counts() for i in range(3)]

assert len(all_counts) == 3, "❌ Challenge 3 failed: should have 3 count dicts!"
assert '0' in all_counts[0] or '1' in all_counts[0], "❌ Challenge 3: circuit 0 wrong"
assert all_counts[1].get('1', 0) == 100, "❌ Challenge 3: circuit 1 should be all 1s"
assert all_counts[2].get('0', 0) == 100, "❌ Challenge 3: circuit 2 should be all 0s"
print(f"   ✅ Circuit 0: {all_counts[0]}")
print(f"   ✅ Circuit 1: {all_counts[1]}")
print(f"   ✅ Circuit 2: {all_counts[2]}")

# ══════════════════════════════════════════════════════════════════════════════
# CHALLENGE 4: Convert counts to probabilities
# ══════════════════════════════════════════════════════════════════════════════
print("\nCHALLENGE 4: Convert counts to probabilities")
qc4 = QuantumCircuit(1)
qc4.h(0)
qc4.measure_all()

job4 = sampler.run([qc4], shots=1000)
result4 = job4.result()
counts4 = result4[0].data.meas.get_counts()

# TODO: Convert to probabilities
total = sum(counts4.values())
probs4 = {k: v/total for k, v in counts4.items()}

assert abs(sum(probs4.values()) - 1.0) < 0.001, "❌ Challenge 4: probs should sum to 1!"
print(f"   ✅ Probabilities: {probs4}")

# ══════════════════════════════════════════════════════════════════════════════
print("\n" + "="*70)
print("🎉 ALL CHALLENGES COMPLETED!")
print("="*70)
print("\n📌 KEY PATTERNS TO MEMORIZE:")
print("   Sampler:   result[0].data.meas.get_counts()  → dict")
print("   Estimator: result[0].data.evs               → float (no parentheses!)")
print("   Multiple:  result[i].data.meas.get_counts() → loop with index")
print("   Probs:     {k: v/total for k, v in counts.items()}")

---

## 📚 Resources

### Conceptual Background
- [README.md](./README.md) - Complete learning guide with mnemonics, traps, practice exam

### One-Page Summary

| Pattern | Code | Returns |
|---------|------|---------|
| **Sampler counts** | `result[0].data.meas.get_counts()` | `dict[str, int]` |
| **Sampler bitstrings** | `result[0].data.meas.get_bitstrings()` | `list[str]` |
| **Sampler int counts** | `result[0].data.meas.get_int_counts()` | `dict[int, int]` |
| **Estimator evs** | `result[0].data.evs` | `float` |
| **Estimator stds** | `result[0].data.stds` | `float` |
| **Metadata** | `result[0].metadata` | `dict` |

### ⚠️ Top Exam Traps
1. Missing `[0]` index: `result.data.meas` ❌ → `result[0].data.meas` ✅
2. Missing `.data`: `result[0].meas` ❌ → `result[0].data.meas` ✅
3. `.evs()` with parentheses: property not method! `result[0].data.evs` ✅
4. Singular `.ev` instead of plural `.evs` ❌
5. Trailing comma: `[(circuit)]` ❌ → `[(circuit,)]` ✅

### 🧠 Mnemonics to Remember
- **RIDMG**: Result → Index → Data → Meas → Get_counts()
- **EVS = plural**: Expectation ValueS (property, no parentheses)
- **IQVRCDE**: JobStatus values in lifecycle order

---

**Notebook verified with Qiskit 1.x** | Last updated: December 2025